In [1]:
import numpy as np

import pysixtrack
import sixtracklib

In [2]:
def get_sixtracklib_particle_set(init_physical_coordinates: np.ndarray, p0c_ev: float):
    n_particles = init_physical_coordinates.shape[0]
    ps = sixtracklib.ParticlesSet()
    p = ps.Particles(num_particles=n_particles)

    for i_part in range(n_particles):
        part = pysixtrack.Particles(p0c=p0c_ev)

        part.x = init_physical_coordinates[i_part, 0]
        part.px = init_physical_coordinates[i_part, 1]
        part.y = init_physical_coordinates[i_part, 2]
        part.py = init_physical_coordinates[i_part, 3]
        part.tau = init_physical_coordinates[i_part, 4]
        part.ptau = init_physical_coordinates[i_part, 5]

        part.partid = i_part
        part.state = 1
        part.elemid = 0
        part.turn = 0

        p.from_pysixtrack(part, i_part)

    return ps

In [3]:
from cpymad.madx import Madx

madx = Madx(stdout=False)
madx.call("lhc_colin_track.seq")
madx.use("lhcb1")  # if not called, no beam present in sequence???
_ = madx.twiss()  # if not called, cavities don't get a frequency???

In [4]:
# Line and sixtracklib Elements
line = pysixtrack.Line.from_madx_sequence(madx.sequence["lhcb1"])

# Only the coordinates at the end of tracking. To keep coordinates at each
# turn, give "num_stores=nturns" when creating the BeamMonitor element
_ = line.append_element(pysixtrack.elements.BeamMonitor(num_stores=1, is_rolling=True), "turn_monitor")
elements = sixtracklib.Elements.from_line(line)

In [5]:
# CO, Linear_OTM and W
closed_orbit, linear_otm = line.find_closed_orbit_and_linear_OTM(
    p0c=madx.sequence["lhcb1"].beam.pc * 1e9, longitudinal_coordinate="tau"
)
W, invW, R = line.linear_normal_form(linear_otm)

Closed orbit search iteration: 0
Closed orbit search iteration: 1
Converged with approximate distance: 2.7900680536855902e-16
Symplectifying linear One-Turn-Map...
Before symplectifying: det(M) = 1.000139999286489
After symplectifying: det(M) = 0.9999999999999987


## Create distribution
---

In [6]:
n_parts = 1_000
energy = 6500
geometric_emittance = 3.75e-6 / (energy / 0.938);

In [7]:
# Get normalized coordinates yolo style
x_normalized = list(np.random.normal(0, np.sqrt(geometric_emittance), n_parts))
px_normalized = list(np.random.normal(0, np.sqrt(geometric_emittance), n_parts))
y_normalized = list(np.random.normal(0, np.sqrt(geometric_emittance), n_parts))
py_normalized = list(np.random.normal(0, np.sqrt(geometric_emittance), n_parts))
tau_normalized = list(np.random.normal(0, 125 * np.sqrt(geometric_emittance), n_parts))
ptau_normalized = list(np.random.normal(0, 125 * np.sqrt(geometric_emittance), n_parts))
normalized_coordinates = np.array(
    [x_normalized, px_normalized, y_normalized, py_normalized, tau_normalized, ptau_normalized]
)

In [8]:
# Go from normalized to physical and center on closed orbit
initial_coordinates = (W @ normalized_coordinates).T
initial_coordinates += closed_orbit

# make sure that stdev of tau (~ bunch length) is correct, should be around 0.08 meters
print(initial_coordinates[:, 4].std())
# the error is stdev / sqrt(2 * n_part)
print(initial_coordinates[:, 4].std() / np.sqrt(2 * n_parts))

0.08250854254800428
0.0018449470986177127


## Prepare Job & Track 
---

In [17]:
particle_set = get_sixtracklib_particle_set(
    init_physical_coordinates=initial_coordinates, p0c_ev=madx.sequence["lhcb1"].beam.pc * 1e9
)
particle_set.to_file("here.particleset")

In [18]:
# Or from dumped file
# particle_set = sixtracklib.ParticlesSet.fromfile("here.particleset")

In [19]:
sixtracklib.TrackJob.print_nodes("opencl")

Device ID Str  : 0.0       
Architecture   : opencl    
Platform       : Apple     
Name           : Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz

Device ID Str  : 0.1       
Architecture   : opencl    
Platform       : Apple     
Name           : Intel(R) UHD Graphics 630

Device ID Str  : 0.2       
Architecture   : opencl    
Platform       : Apple     
Name           : AMD Radeon Pro Vega 20 Compute Engine


In [20]:
# For GPU use, specify GPU device_id from information given by clinfo in the cell above
job = sixtracklib.TrackJob(elements, particle_set, device_id="opencl:0.2")
job.track_until(100)
job.collect()  # transfer data back from GPU, fine to call if CPU only

In [21]:
final_output = {
    "x": job.output.particles[0].x,
    "px": job.output.particles[0].px,
    "y": job.output.particles[0].y,
    "py": job.output.particles[0].py,
    "zeta": job.output.particles[0].zeta,
    "delta": job.output.particles[0].delta,
    "at_turn": job.output.particles[0].at_turn,
}

In [22]:
final_output

{'x': array([ 1.20312332e-05,  1.24851551e-04,  8.59765201e-05, -5.58244696e-04,
        -1.43928508e-04,  1.32702396e-04, -5.73442379e-04,  4.15455607e-04,
         7.60457766e-05,  3.29856984e-04, -2.61768103e-04,  3.30548778e-04,
        -7.69830730e-05,  1.33486262e-04,  2.44390943e-04,  2.31344430e-04,
        -2.42884934e-06, -2.80942419e-04, -1.24553101e-04,  5.74523157e-04,
         2.11826108e-04, -5.17849406e-04,  6.65361085e-05, -2.47715166e-04,
        -6.79813130e-05,  7.72405412e-05,  8.89561473e-05, -2.90160131e-04,
        -2.04169090e-04,  3.21206555e-05, -1.50351490e-04,  4.51935274e-04,
        -4.76456055e-05,  4.17309757e-04,  1.33979591e-04,  2.64723118e-04,
         3.66091008e-04, -2.43805930e-04, -2.16082039e-04, -3.93341667e-04,
        -2.65453583e-04,  1.02145411e-04, -7.47697201e-05,  7.87536721e-05,
         3.10403074e-04,  1.14170403e-04, -3.21155711e-04, -6.69112529e-06,
         4.88898921e-04,  3.47435059e-05, -2.35825148e-05, -1.06456262e-04,
       

In [15]:
# trick to get tau and ptau: define a Particle with these results in pysixtrack and get the results from there
p = pysixtrack.Particles(p0c=madx.sequence["lhcb1"].beam.pc * 1e9)
p.zeta = final_output["zeta"]
p.delta = final_output["delta"]
p.tau

array([ 2.04749894e-01,  2.39639807e-02, -8.05628462e-02, -1.06675433e-01,
        3.15907161e-03, -7.60809539e-02,  1.82573157e-02, -6.91492410e-02,
       -2.89677697e-02,  6.30341300e-02, -3.26014516e-02, -8.66578884e-03,
        3.33953697e-02, -9.71710422e-02, -1.28480787e-01,  7.94923209e-02,
        1.34704801e-01,  7.12863253e-02, -4.70001200e-03,  3.83297898e-02,
       -8.45318427e-02, -4.66510201e-02,  3.47445780e-03,  3.85871736e-02,
       -7.38239387e-02,  9.00524298e-02,  3.01339094e-02, -1.04487805e-03,
        1.12467166e-04, -6.59547111e-02, -2.05029413e-02, -6.24671553e-02,
        5.09816910e-02,  8.76230019e-02,  8.39363928e-02, -3.59660599e-02,
       -5.16672727e-02, -3.71237431e-02,  2.25860227e-02, -1.07318203e-01,
       -8.98272642e-02,  1.95060984e-01, -4.74566703e-02, -1.11443521e-02,
        5.25021566e-02,  9.76005096e-02, -1.81196977e-01, -9.33891925e-03,
       -8.32097688e-02, -8.43209295e-02, -8.29086065e-02, -7.40714870e-02,
        4.04282127e-02, -

---